### 1. Enviroment set up

In [1]:
# Libraries
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
# Define the database parameters

db_params = {
    'host' : 'localhost',
    'user' : 'postgres',
    'password' : 'postgres',
    'database' : 'postgres'
}

### 2. Creating the postgresql database

In [3]:
# Create a connection to the postgresql database

connection = psycopg2.connect(
    host = db_params['host'],
    database = db_params['database'],
    user = db_params['user'],
    password = db_params['password']
)

# Create a cursor object, allows me to execute sql queries
cursor = connection.cursor()

# set automatic commit to true, so each action is committed without having to call con.commit()
connection.set_session(autocommit = True)

# Create the 'retail' database
cursor.execute('CREATE DATABASE retail')

# Close connection to postgres database, leaving connection to retail database open and commit changes
connection.commit()
cursor.close()
connection.close()


### 3. Loading the data to transform it

In [23]:
# import data
data = pd.read_csv('data/Online Retail.csv')

data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2010 8:26,3.39,17850.0,United Kingdom


In [24]:
# change the column names to make sql queries easier
new_columns = ['invoice_no', 'stock_code', 'description','quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
data.columns = new_columns

In [25]:
data['invoice_date'] = pd.to_datetime(data['invoice_date'], format = 'mixed')
data.reset_index(drop = True, inplace = True)
data.head(5)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom


### 4. Loading the data to a new table called 'retail' in the retail database

In [26]:
# Connect to the retail database
db_params['database'] = 'retail'
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')


# Load the data to the database
data.to_sql(name = 'retail', con = engine, if_exists = 'replace', index = 'False')

909